<h1 style="color:#2c3f51">Predict the likelihood of a genome sequence undergoing mutation.</h1>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import optuna

# Metrics
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [2]:
train = pd.read_csv('../input/devday22-competition-datascience/train.csv')
test = pd.read_csv('../input/devday22-competition-datascience/test.csv')

print('Data Dimension\n')
train.shape, test.shape

Data Dimension



((180000, 32), (120000, 31))

# Understanding Data

- Data describe

In [3]:
train.describe()

,ID,A,B,C,D,E,F,G,H,I,...,M,N,O,P,Q,R,S,T,U,mutation
count,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,...,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000
mean,150259.572350,8.118989,2.632989,0.617833,4.802733,29.696178,1.262467,23.543444,33.423717,1.605150,...,0.516286,0.474217,0.505086,0.501836,0.487837,0.501700,0.487640,0.469766,0.507951,0.264900
std,86591.625009,3.078097,4.147355,1.191218,1.514448,12.700896,2.109440,13.897618,18.991241,2.752459,...,0.214696,0.216486,0.227391,0.241277,0.211201,0.203394,0.178990,0.194466,0.203378,0.441281
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.099276,-0.045316,0.169495,-0.033695,0.005199,0.093056,0.024139,0.215575,0.097789,0.000000
25%,75313.750000,7.000000,0.000000,0.000000,4.000000,33.000000,0.000000,9.000000,19.000000,0.000000,...,0.325804,0.291995,0.279693,0.276133,0.323797,0.353174,0.358795,0.310106,0.367429,0.000000
50%,150321.000000,8.000000,0.000000,0.000000,4.000000,33.000000,0.000000,23.000000,38.000000,0.000000,...,0.471377,0.388573,0.480476,0.554751,0.517962,0.435187,0.410841,0.408567,0.446227,0.000000
75%,225293.250000,10.000000,3.000000,1.000000,5.000000,33.000000,2.000000,35.000000,49.000000,2.000000,...,0.704146,0.645087,0.725755,0.735208,0.607613,0.642035,0.611452,0.587050,0.581691,1.000000
max,299999.000000,14.000000,18.000000,12.000000,19.000000,83.000000,15.000000,50.000000,60.000000,18.000000,...,1.016600,0.951018,0.856975,0.853022,0.960912,1.035818,1.054257,1.005652,1.011331,1.000000


In [4]:
test.describe()

,ID,A,B,C,D,E,F,G,H,I,...,L,M,N,O,P,Q,R,S,T,U
count,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,...,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000
mean,149609.391475,8.127517,2.616475,0.612867,4.811917,29.675250,1.261375,23.459275,33.394692,1.607508,...,0.494413,0.517149,0.474261,0.504490,0.502889,0.488817,0.501796,0.488725,0.469092,0.508648
std,86618.168318,3.060814,4.119720,1.173331,1.520519,12.679975,2.123597,13.852400,19.019033,2.770016,...,0.213312,0.215083,0.216861,0.227599,0.241192,0.211534,0.203650,0.179134,0.194591,0.203416
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.084480,0.094493,-0.031298,0.168071,-0.036379,0.018907,0.090901,0.045832,0.214866,0.126711
25%,74540.750000,7.000000,0.000000,0.000000,4.000000,33.000000,0.000000,9.000000,19.000000,0.000000,...,0.317640,0.326394,0.292660,0.278682,0.276907,0.324116,0.352900,0.359112,0.309691,0.369417
50%,149557.500000,8.000000,0.000000,0.000000,4.000000,33.000000,0.000000,23.000000,38.000000,0.000000,...,0.462719,0.471137,0.385824,0.479183,0.554798,0.518270,0.435730,0.411012,0.405421,0.447079
75%,224504.250000,10.000000,3.000000,1.000000,5.000000,33.000000,2.000000,35.000000,49.000000,2.000000,...,0.695031,0.705315,0.644702,0.725804,0.735256,0.609925,0.641976,0.614888,0.586463,0.582367
max,299992.000000,14.000000,18.000000,12.000000,19.000000,83.000000,15.000000,50.000000,60.000000,18.000000,...,1.005793,1.005743,0.952187,0.858578,0.846413,0.966553,1.027458,1.055885,1.005392,1.006479


- Data info

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 32 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ID         180000 non-null  int64  
 1   sequence1  180000 non-null  object 
 2   sequence2  180000 non-null  object 
 3   sequence3  180000 non-null  object 
 4   sequence4  180000 non-null  object 
 5   sequence5  180000 non-null  object 
 6   sequence6  180000 non-null  object 
 7   sequence7  180000 non-null  object 
 8   sequence8  180000 non-null  object 
 9   sequence9  180000 non-null  object 
 10  A          180000 non-null  int64  
 11  B          180000 non-null  int64  
 12  C          180000 non-null  int64  
 13  D          180000 non-null  int64  
 14  E          180000 non-null  int64  
 15  F          180000 non-null  int64  
 16  G          180000 non-null  int64  
 17  H          180000 non-null  int64  
 18  I          180000 non-null  int64  
 19  J          180000 non-n

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 31 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ID         120000 non-null  int64  
 1   sequence1  120000 non-null  object 
 2   sequence2  120000 non-null  object 
 3   sequence3  120000 non-null  object 
 4   sequence4  120000 non-null  object 
 5   sequence5  120000 non-null  object 
 6   sequence6  120000 non-null  object 
 7   sequence7  120000 non-null  object 
 8   sequence8  120000 non-null  object 
 9   sequence9  120000 non-null  object 
 10  A          120000 non-null  int64  
 11  B          120000 non-null  int64  
 12  C          120000 non-null  int64  
 13  D          120000 non-null  int64  
 14  E          120000 non-null  int64  
 15  F          120000 non-null  int64  
 16  G          120000 non-null  int64  
 17  H          120000 non-null  int64  
 18  I          120000 non-null  int64  
 19  J          120000 non-n

- Check missing values

In [7]:
train.isnull().sum().any()

False

In [8]:
test.isnull().sum().any()

False

- Feature correlation

In [9]:
#train.corr

# Label encode

We are going to encode objects columns

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Object columns selection

In [11]:
obj_cols = [col for col in train.columns if train[col].dtype == 'object']
obj_cols

['sequence1',
 'sequence2',
 'sequence3',
 'sequence4',
 'sequence5',
 'sequence6',
 'sequence7',
 'sequence8',
 'sequence9']

Encoding label

In [12]:
for col in obj_cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# Split Data

In [13]:
# training Data
X = train.drop(['ID','mutation'], axis=1)
y = train['mutation']

# Test data
test = test.drop(['ID'], axis=1)

X.shape, test.shape

((180000, 30), (120000, 30))

# Choose a model

Here, we will compare the performance of three algos. We will use :
1. CatBoostClassifier : [doc here](https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier)
2. XGBClassifier : [doc here](https://xgboost.readthedocs.io/en/stable/python/python_api.html)
3. LGBMClassifier : [doc here](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html)

In [14]:
# Import labraries
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [15]:
models = [
    CatBoostClassifier(verbose=False), 
    XGBClassifier(), 
    LGBMClassifier()
]

In [16]:
for model in models:
    # Fit model
    model.fit(X,y)  
    
    # Test Model
    predict = model.predict(X)              
    
    # Evaluate model
    print(f"Accuracy score ({str(model)}) : ", accuracy_score(y, predict), '\n')

Accuracy score (<catboost.core.CatBoostClassifier object at 0x7f725c2bde90>) :  0.86995 

Accuracy score (XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)) :  0.8688166666666667 

Accuracy score (LGBMClassifier()) :  0.85205 



### Free the memory

In [17]:
import gc

del models, train, le, obj_cols
gc.collect()

139

# Modeling with CatBoost

Here, we'll use optuna to search the best params of catboost model 

In [18]:
def objectiveCatboost(trial):
    iterations = trial.suggest_int("iterations", 10, 3000)
    max_depth = trial.suggest_int("max_depth", 1, 16)
    random_state = trial.suggest_int("random_state", 1, 2022)
    learning_rate = trial.suggest_float("learning_rate", 0, 1)
    verbose = False
    
    clf = CatBoostClassifier(iterations = iterations,
                               max_depth = max_depth,
                               learning_rate = learning_rate,
                               random_state = random_state,
                               verbose=verbose
                              )
    clf.fit(X, y)
    return clf.score(X, y)

"""
study = optuna.create_study(direction = "maximize")
study.optimize(objectiveCatboost, n_trials=12)
"""

'\nstudy = optuna.create_study(direction = "maximize")\nstudy.optimize(objectiveCatboost, n_trials=12)\n'

The best params is :

In [19]:
#parameters = study.best_params
parameters = {'iterations': 2035, 'max_depth': 12, 'random_state': 1800, 'learning_rate': 0.34696069649515215}

# Cross validation

In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Test prediction

In [21]:
kf = StratifiedKFold(n_splits=7, shuffle=True, random_state = 0)
y_predict = []

for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        print('*'*14, f" Fold {fold} ", '*'*14, '\n')
        
        # Split Data
        X_train = X.loc[train_index]
        X_val = X.loc[val_index]

        y_train = y.loc[train_index]
        y_val = y.loc[val_index]
        
        # Create Model here
        model = CatBoostClassifier(**parameters, verbose=0)
        
        # Fit Model
        model.fit(X_train,y_train)

        # Make X_val prediction
        y_pred = model.predict_proba(X_val)[:,1]
        
        # Make Test prediction
        test_pred = model.predict_proba(test)[:,1]
        y_predict.append(test_pred)
        
        # Evaluate Model
        #print("Training score :", model.score(X_train, y_train))
        print("Roc auc score  :", roc_auc_score(y_val, y_pred), '\n')
        
        # Free the memory
        del X_train, y_train, model, X_val, y_val, y_pred, test_pred
        gc.collect()


**************  Fold 0  ************** 

Roc auc score  : 0.8713766442886139 

**************  Fold 1  ************** 

Roc auc score  : 0.8716668190346185 

**************  Fold 2  ************** 

Roc auc score  : 0.8738715242239292 

**************  Fold 3  ************** 

Roc auc score  : 0.8680151454854176 

**************  Fold 4  ************** 

Roc auc score  : 0.8691110942598326 

**************  Fold 5  ************** 

Roc auc score  : 0.8757968364565186 

**************  Fold 6  ************** 

Roc auc score  : 0.8712203992121057 



# Submission

In [22]:
submission = pd.read_csv('../input/devday22-competition-datascience/sample_submission.csv')
submission.head()

,ID,mutation
0,1,0.5
1,3,0.5
2,4,0.5
3,6,0.5
4,8,0.5


In [23]:
submission['mutation'] = np.array(y_predict).mean(axis=0)
submission.to_csv('submission.csv', index=False)
submission

,ID,mutation
0,1,0.429409
1,3,0.002682
2,4,0.141149
3,6,0.015162
4,8,0.000459
...,...,...
119995,299983,0.010378
119996,299984,0.913129
119997,299990,0.847451
119998,299991,0.000128


<center>
    <h2 style="color:#2c3f51">Thanks for reading...</h2>